**Insper**  
**Análise de Textos de Fontes Desestruturadas e WEB**

# **IMDB:<br/>Recomendação de filmes baseada em semelhança de tópicos**

**Beatriz de Jesus**  
**Luciano Felix**  
**Rodrigo Villela**


## **1. Introdução**

O projeto que estamos desenvolvendo tem como objetivo criar uma ferramenta de recomendação de filmes personalizada, capaz de sugerir filmes com base nas preferências do usuário. Utilizando técnicas de webscraping e feature engineering, vamos extrair informações sobre filmes no IMDb. Essas informações incluirão título, rating médio dos usuários e enredo. Com base nesses dados, aplicaremos o algoritmo de recomendação para oferecer sugestões de filmes que sejam compatíveis com o interesse do usuário.


## **2. Problema**

É muito comum a dificuldade que as pessoas encontram tem em escolher o que assistir em seu momento de lazer. Ironicamente, ter tantas opções nos torna mais seletivos e gera grande dificuldade para filtrar o que aparenta ser a melhor opção. O objetivo do projeto é criar um algoritmo capaz de identificar a melhor opção para o usuário a partir de uma rápida descrição deste mesmo. O programa fará isso captando as ‘keywords’ atreladas ao filme e correlacionando-as com a descrição dada pelo usuário.


## **3. Extração de Dados**

In [ ]:
!pip install ipywidgets

In [ ]:
from dataclasses import dataclass
from typing import List, Dict
from urllib.parse import urlparse
from pathlib import PurePath

import requests
import numpy as np
import pandas as pd
from ast import literal_eval
from bs4 import BeautifulSoup as bs
import ipywidgets as widgets

import nltk
from gensim import corpora
from gensim import models
from gensim.parsing.preprocessing import preprocess_string

nltk.download('stopwords')
nltk.download('rslp')


BASE_REQUESTS_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
}


### **3.1. Obtenção da base de índice dos filmes**

In [ ]:
def get_top_250_ids(headers: Dict[str, str] = BASE_REQUESTS_HEADERS) -> List[str]:
    url = "https://www.imdb.com/chart/top"
    response = requests.get(url, headers=headers)
    soup = bs(response.text)

    return [
        PurePath(urlparse(node.attrs["href"]).path).name
        for node in soup.select('table > tbody > tr > td.titleColumn > a')
    ]

top_250_ids = get_top_250_ids()
top_250_ids[:5]

### **3.2. Obtenção dos dados de cada filme individualmente**

In [ ]:
@dataclass
class Title:
    id: str
    title: str
    score: float
    plot: str
    keywords: str

def fetch_title_keywords(title_id: str, headers: Dict[str, str] = BASE_REQUESTS_HEADERS) -> List[str]:
    url = f"https://www.imdb.com/title/{title_id}/keywords"
    response = requests.get(url, headers=headers)
    soup = bs(response.text)

    item_nodes = soup.select(".ipc-metadata-list-summary-item a")

    return [item.text for item in item_nodes]


def fetch_title(title_id: str, headers: Dict[str, str] = BASE_REQUESTS_HEADERS) -> Title:
    url = f"https://www.imdb.com/title/{title_id}/"
    response = requests.get(url, headers=headers)
    soup = bs(response.text)

    title_node = soup.find("h1")
    score_node = soup.select_one('[data-testid="hero-rating-bar__aggregate-rating__score"]').next_element
    plot_node = soup.select_one('[data-testid="plot-xl"]')

    return Title(
        id = title_id,
        title = title_node.text,
        score = float(score_node.text.replace(",", ".")),
        plot = plot_node.text,
        keywords = " ".join(fetch_title_keywords(title_id))
    )


In [ ]:
titles = []
for index, title_id in enumerate(top_250_ids, 1):
    print(f"\r{index} of {len(top_250_ids)}", end="")

    titles.append(fetch_title(title_id) )

print(f"\rComplete! ")

df = pd.DataFrame(titles)
df.head(5)

### **3.3 Limpeza dos dados**

In [ ]:
df["tokens"] = (df["plot"]+df["keywords"]).map(preprocess_string)

df.head(5)

### **3.4. Salvando a base localmente**

In [ ]:
df.to_csv("./project/data/imdb_top_250.csv")

## **4. Análise**

In [ ]:
df = pd.read_csv("./project/data/imdb_top_250.csv", index_col=0)

df["tokens"] = df["tokens"].apply(literal_eval)

df.head()

### **4.1. Construção do modelo**

In [ ]:
tokens = df["tokens"]

dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(document) for document in tokens]

In [ ]:
lda = models.ldamodel.LdaModel(
    corpus = corpus,
    id2word=dictionary,
    num_topics=df.shape[0],
    passes=25,
    random_state=1,
    iterations=100
)

lda.save("./project/data/model.gensim")

### **4.2. Extração dos tópicos da base de dados**

In [ ]:
lda = lda.load("./project/data/model.gensim")

In [ ]:
def get_best_topic_match(topics):
    topics_index, topics_chance = list(zip(*topics))

    return topics[np.argmax(topics_chance)]


topic_index = []
accuracy = []

for index, row in df.iterrows():
    corpus = dictionary.doc2bow(row["tokens"])
    topics = lda.get_document_topics(corpus)
    best_match = get_best_topic_match(topics)

    topic_index.append(best_match[0])
    accuracy.append(best_match[1])

df["topic"] = topic_index
df["topic_accuracy"] = accuracy

df.head()

## **4.3. Encontrando correspodentes bom base nos tópicos**

Insira na caixa de texto de descrição, as características do filme que você busca. Por exemplo:

> After being forced to work for a crime boss, a young driver becomes involved in a heist destined to fail.

Fonte: [Em Ritmo de Fuga, IMDb](https://www.imdb.com/title/tt3890160/).

In [249]:
topics_input = widgets.Text(
    description="Descrição:",
    layout=widgets.Layout(
        width='auto'
    )
)

topics_input

Text(value='', description='Descrição:', layout=Layout(width='auto'))

In [248]:
@dataclass
class Title:
    accuracy: str
    title: str
    score: float
    link: str

input_tokens = preprocess_string(topics_input.value)
input_corpus = dictionary.doc2bow(input_tokens)
input_topics = lda.get_document_topics(input_corpus)
topics = lda.show_topics(num_topics=-1, num_words=len(input_corpus), formatted=False)
titles = []

for index, accuracy in input_topics:
    matches = np.where(df["topic"] == index)[0]
    movies = df.iloc[matches, :]

    for _, movie in movies.iterrows():
        match_accuracy = accuracy * movie["topic_accuracy"]

        title = Title(
            accuracy=f"{match_accuracy*100:.2f}%",
            title=movie["title"],
            score=movie["score"],
            link=f'https://www.imdb.com/title/{movie["id"]}',
        )

        titles.append(title)

pd.DataFrame(titles).sort_values("accuracy", ascending=False)

,accuracy,title,score,link
8,42.78%,Taxi Driver: Motorista de Táxi,8.2,https://www.imdb.com/title/tt0075314
9,42.74%,Três Anúncios para um Crime,8.1,https://www.imdb.com/title/tt5027774
10,42.74%,Relatos Selvagens,8.1,https://www.imdb.com/title/tt3011894
7,42.67%,O Senhor dos Anéis: A Sociedade do Anel,8.8,https://www.imdb.com/title/tt0120737
0,18.57%,O Poderoso Chefão,9.2,https://www.imdb.com/title/tt0068646
3,18.56%,A Mulher Faz o Homem,8.1,https://www.imdb.com/title/tt0031679
2,18.53%,Sindicato de Ladrões,8.1,https://www.imdb.com/title/tt0047296
5,16.10%,Fogo Contra Fogo,8.3,https://www.imdb.com/title/tt0113277
4,15.21%,Os Incríveis,8.0,https://www.imdb.com/title/tt0317705
1,14.33%,Bastardos Inglórios,8.3,https://www.imdb.com/title/tt0361748


## **5. Conclusão**

Ficamos surpreendidos como um modelo simples é capáz de criar sugestões bastante precisas. E caso tivésse-mos uma base mais ampla, o quê não seria viável de adquirir no curto prazo, teríamos uma aplicação muito mais útil.

Apesar de uma base de dados limitada, foi possível fazer um programa eficiente, capaz de encontrar bons resultados para a entrada do usuário. A base de dados também é enviesada por listar os 250 filmes mas bem avaliados da plataforma do IMDb, então caso o usuário insira a sinopse de um filme que não está na base, por exemplo, o programa irá sugerir diversos filmes com baixa acurácia.